In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import datetime
#turns seconds into a string format ffmpeg uses
def seconds_formatter(sec):
    x = "0" + str(datetime.timedelta(seconds=sec))[:-3]
    return x

## Random Forest Classifier
identify reaches, successes, failures

# Project Variables 
### Change as needed.

In [147]:
df = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos\A_copiedDeepCut_resnet50_goprolensfrontJun25shuffle1_60002.h5")
#remove extra level "scorer"
df.columns = df.columns.droplevel()

frame_rate = 119.88

total_num_frames = df.shape[0]

cols = ['Nose', 'DomInside', 'Pellet', 'Index', 'Middle', 'Ring', 'DomOutside', 'OtherHand']

classes = ['n', 's', 'f', 'r']

frames_before = 12

frames_after = 24

step_size = 3

seed = 74

In [148]:
def frame_to_time(frame):
    return round(frame/frame_rate, 3)
def time_to_frame(time):
    return int(round(time*frame_rate))

### Functions for Creating Classifier

In [149]:
import random
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier

#Functions to get training data
def get_dataset(df, start_frames):
    print(df.shape)
    dataframe = df[cols]
    dataset = []
    for start_f in start_frames:
        temp_df = dataframe.iloc[start_f - frames_before: start_f + frames_after]
        if len(temp_df.values.flatten()) == 0:
            print(start_f)
            print(dataframe.shape)
        dataset.append(temp_df.values.flatten())
    return np.asarray(dataset)
def get_non_reach_frames(df, num_frames, blacklist, avoidance_radius):
    non_reaches = []
    random.seed(seed)
    while len(non_reaches) < num_frames:
        #hard code 3 non-reaches
        frame = random.randint(frames_before+10, df.shape[0] - frames_after)

        too_close = False
        for start_time in blacklist:
            if abs(frame - start_time) < avoidance_radius:
                print('no')
                too_close = True
                break
        if not too_close:
            non_reaches.append(frame)
    return non_reaches

#Use training features and labels to create Random Forest Classifier
def create_classifier(features, labels, n_trees, test_proportion=0.25):
    train_feats, test_feats, train_labs, test_labs = train_test_split(features, labels, test_size = test_proportion, random_state = seed)
    print('Training Features Shape:', train_feats.shape)
    print('Training Labels Shape:', train_labs.shape)
    print('Testing Features Shape:', test_feats.shape)
    print('Testing Labels Shape:', test_labs.shape)
    rf = RandomForestClassifier(n_estimators = n_trees, random_state = seed)
    # Train the model on training data
    print(train_feats.shape)
    print(train_labs.shape)
    rf.fit(train_feats, train_labs)
    # Use the forest's predict method on the test data
    predictions = rf.predict(test_feats)
    print(predictions)
    print(test_labs)
    wrong_labels = []
#     for i in range(len(predictions)):
#         if predictions[i] != test_labs[i]:
#             frame = -1
#             for key in frame_to_features.keys():
#                 if (frame_to_features[key] == test_feats[i]).all():
#                     frame = key
#                     break
#             wrong_labels.append((frame_to_time(frame), predictions[i], test_labs[i]))
    for i in range(len(predictions)):
        if predictions[i] != test_labs[i]:
            frame = -1
            wrong_labels.append((predictions[i], test_labs[i]))
    print('Wrong Labels', wrong_labels)
    return rf, wrong_labels

### Training Data

In [150]:
# all_frames = reach_framesA+non_reaches_framesA + reach_frames_n2v3+non_reaches_frames_n2v3 + reach_frames_n3v3+non_reaches_frames_n3v3 + reach_frames_injuredn2v3+non_reaches_frames_injuredn2v3
# vid_labels = ['A' for x in labelsA] + ['n2v3' for x in labels_n2v3] + ['n3v3' for x in labels_n3v3] + + ['injuredn2v3' for x in labels_injuredn2v3]
# frame_to_features = dict(zip(list(zip(all_frames, vid_labels)), compiled_features))

# print(len(all_frames))
#"C:\Users\vjj14\Documents\GoProLens\3lens\gp2.7k3lensA1.mp4"
df_A = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\gopro3lens-vj-2019-07-08\videos\gp2.7k3lensA1DeepCut_resnet50_gopro3lensJul8shuffle1_120001.h5")
df_A.columns = df_A.columns.droplevel()
reach_timesA = [19.12, 23.752, 24.186, 24.630, 36.846, 43.652, 44.042, 46.964, 60.218, 62.897, 64.787, 72.045, 85.174, 88.196, 
           91.652, 93.248, 93.814, 106.294, 106.867, 111.344, 112.249, 113.407, 115.474, 115.918, 129.119, 136.837, 138.581]
reach_framesA = [time_to_frame(x) for x in reach_timesA]
reach_labelsA = ['r', 'f', 'r', 'r', 's', 'f', 's', 'r', 's', 'r', 'r', 'r', 's', 'r', 's', 'r', 'r', 'r', 'f', 'r', 'r', 
                      'r', 'f', 'r', 's', 'r', 'r']
non_reaches_framesA = get_non_reach_frames(df_A, len(reach_framesA)//2, reach_framesA, 12)
featuresA = get_dataset(df_A, reach_framesA+non_reaches_framesA)
labelsA = np.asarray(reach_labelsA + ['n' for x in non_reaches_framesA])


#"D:\DCIM\100GOPRO\2019.07.08_CC42970_N2_V3_2.7k_120FPS_Label_copy.mp4"
df_n2v3 = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\gopro3lens-vj-2019-07-08\videos\2019.07.08_CC42970_N2_V3_2.7k_120FPS_Label_copyDeepCut_resnet50_gopro3lensJul8shuffle1_120001.h5")
df_n2v3.columns = df_n2v3.columns.droplevel()
reach_times_n2v3 = [12.325, 16.568, 17.55, 18.912, 20.41, 22.1, 22.92, 24.699, 27.4, 29.754, 30.431, 31.39, 32.692, 37.731, 
           38.258, 42.86, 45.714, 56.45, 76.803, 77.86, 86.2, 86.75, 91.13, 91.649, 97.563, 98.1, 106.739, 107.292, 116.412, 
           121.49, 122.236, 124.775, 125.375, 134.324, 134.859, 142.37, 143.0, 150.0, 159.435, 160.04, 165.02, 
           165.633, 174.018, 177.129, 179.23, 183.719, 185.755, 192.041, 197.214, 198.66, 201.286, 203.34, 204.15, 206.253, 
           209.36, 213.027, 213.895, 217.676, 218.325, 218.982, 223.198, 226.56, 230.524, 233.61, 234.286, 235.838, 236.316, 
           237.091, 250.448, 251.206, 253.05]
reach_frames_n2v3 = [time_to_frame(x) for x in reach_times_n2v3]
reach_labels_n2v3 = ['s', 'r', 'f', 'r', 'r', 'f', 'r', 'r', 'r', 'f', 'r', 'r', 'r', 'f', 'r', 'r', 's', 'r', 'f', 'r', 
                  'f', 'r', 'f', 'r', 'f', 'r', 'f', 'r', 'f', 'r', 'r', 'f', 'r', 'f', 'r', 'f', 'r', 'f', 'f', 'r', 
                  'f', 'r', 's', 'r', 's', 'f', 'r', 's', 'f', 'r', 'f', 'r', 'r', 's', 'r', 'f', 'r', 'r', 'f', 'r', 's', 
                  'r', 's', 'r', 'r', 'f', 'f', 'r', 'f', 'r', 'r']
non_reaches_frames_n2v3 = get_non_reach_frames(df_n2v3, len(reach_frames_n2v3)//2, reach_frames_n2v3, 12)
features_n2v3 = get_dataset(df_n2v3, reach_frames_n2v3+non_reaches_frames_n2v3)
labels_n2v3 = np.asarray(reach_labels_n2v3 + ['n' for x in non_reaches_frames_n2v3])
print(len(labels_n2v3))
print(len(features_n2v3))
print(len(reach_frames_n2v3))
print(len(non_reaches_frames_n2v3))

#"D:\DCIM\100GOPRO\2019.07.08_CC42970_N3_V3_2.7k_120FPS_Label_copy.mp4"
df_n3v3 = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\gopro3lens-vj-2019-07-08\videos\2019.07.08_CC42970_N3_V3_2.7k_120FPS_Label_copyDeepCut_resnet50_gopro3lensJul8shuffle1_120001.h5")
df_n3v3.columns = df_n3v3.columns.droplevel()
reach_times_n3v3 = ['11.589', '17.755', '18.474', '20.810', '22.457', '25.491', '37.403', '41.884', '51.558', '57.359', 
                         '64.503', '78.54', '84.611', '96.276', '108.619', '117.017', '123.131', '130.131', '157.557', 
                         '164.342', '207.559']
reach_frames_n3v3 = [time_to_frame(float(x)) for x in reach_times_n3v3]
reach_labels_n3v3 = ['s', 'f', 'r', 'r', 'r', 's', 'f', 's', 'f', 's', 'f', 'f', 's', 'f', 's', 's', 'f', 'f', 's', 
                          's', 'f']
non_reaches_times_n3v3 = [3.333, 126.886, 133.355, 205.494, 77.901, 94.947, 26.119, 73.896, 166.391, 51.277]
non_reaches_frames_n3v3 = [time_to_frame(x) for x in non_reaches_times_n3v3]
features_n3v3 = get_dataset(df_n3v3, reach_frames_n3v3+non_reaches_frames_n3v3)
labels_n3v3 = np.asarray(reach_labels_n3v3 + ['n' for x in non_reaches_frames_n3v3])

#"D:\DCIM\100GOPRO\2019.07.08_CC42973_N2INJURED_V3_2.7k_120FPS_Label.MP4"
df_injuredn2v3 = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\gopro3lens-vj-2019-07-08\videos\2019.07.08_CC42973_N2INJURED_V3_2.7k_120FPS_Label_copyDeepCut_resnet50_gopro3lensJul8shuffle1_120001.h5")
df_injuredn2v3.columns = df_injuredn2v3.columns.droplevel()
reach_times_injuredn2v3 = ['21.196', '66.492', '71.227', '88.349', '92.221', '99.692', '116.299', '127.703', '135.332', 
                                '142.429', '147.11', '151.609', '157.722', '163.4', '172.798', '183.103', '204.054', '208.735', '216.329']
reach_frames_injuredn2v3 = [time_to_frame(float(x)) for x in reach_times_injuredn2v3]
reach_labels_injuredn2v3 = ['f', 's', 'f', 'f', 'r', 'f', 'f', 's', 'f', 'f', 's', 's', 's', 's', 'f', 'f', 'f', 'f', 's']
non_reaches_times_injuredn2v3 = [157.463, 38.734, 62.743, 66.176, 88.856, 94.266, 171.907, 179.534, 184.588, 207.388]
non_reaches_frames_injuredn2v3 = [time_to_frame(x) for x in non_reaches_times_injuredn2v3]
features_injuredn2v3 = get_dataset(df_injuredn2v3, reach_frames_injuredn2v3+non_reaches_frames_injuredn2v3)
labels_injuredn2v3 = np.asarray(reach_labels_injuredn2v3 + ['n' for x in non_reaches_frames_injuredn2v3])

frame_to_features

(17061, 27)
no
no
(30598, 27)
106
106
71
35
(25677, 27)
(26534, 27)


{29: array([1.32049148e+03, 4.89703739e+01, 6.91568375e-01, ...,
        1.48693424e+03, 6.30204360e+02, 1.08153604e-01]),
 138: array([1.34223232e+03, 8.76056427e+02, 9.19414878e-01, ...,
        1.46849029e+03, 7.90101159e+02, 3.69126469e-01]),
 221: array([1.33864680e+03, 8.92323871e+02, 7.82037616e-01, ...,
        1.44411219e+03, 7.72731573e+02, 9.27546382e-01]),
 1235: array([1.13175907e+03, 6.02675882e+02, 5.81155624e-03, ...,
        1.51987363e+03, 6.52329251e+02, 6.96061030e-02]),
 1365: array([1.10679177e+03, 3.00457683e+02, 6.38255477e-03, ...,
        1.34277055e+03, 9.53990662e+02, 9.96151380e-03]),
 1681: array([1.34167192e+03, 8.66259932e+02, 8.68879616e-01, ...,
        1.50101383e+03, 7.43476717e+02, 4.21648175e-01]),
 1743: array([1.33987860e+03, 8.77755175e+02, 9.61916804e-01, ...,
        1.49165158e+03, 7.40725704e+02, 9.88111138e-01]),
 1885: array([1.33918982e+03, 8.87501074e+02, 9.29724932e-01, ...,
        1.34278034e+03, 9.05434349e+02, 3.65166254e-02]),
 194

In [151]:
compiled_features = np.concatenate((featuresA, features_n2v3, features_n3v3, features_injuredn2v3))
compiled_labels = np.concatenate((labelsA, labels_n2v3, labels_n3v3, labels_injuredn2v3))

In [152]:
attempt_classifier = create_classifier(compiled_features, compiled_labels, 1000, test_proportion=0.01)[0]

Training Features Shape: (203, 864)
Training Labels Shape: (203,)
Testing Features Shape: (3, 864)
Testing Labels Shape: (3,)
(203, 864)
(203,)
['r' 'n' 'f']
['r' 'n' 's']
Wrong Labels [('f', 's')]


In [124]:
wrong_ones = []
for i in range(100, 200):
    seed = i
    wrong_ones.extend(create_classifier(compiled_features, compiled_labels, 1000, test_proportion=0.1)[1])
wrong_ones.sort(key= lambda x: x[0])
wrong_df = pd.DataFrame(wrong_ones, columns = ['guess', 'actual'])
wrong_df.groupby(wrong_df.columns.tolist(),as_index=False).size()

Training Features Shape: (185, 864)
Training Labels Shape: (185,)
Testing Features Shape: (21, 864)
Testing Labels Shape: (21,)
(185, 864)
(185,)
['s' 'f' 'r' 'r' 'f' 'f' 's' 'n' 'n' 'n' 'f' 'f' 'n' 'n' 'f' 'n' 'f' 'n'
 'r' 'n' 'f']
['s' 's' 'r' 'r' 's' 'f' 's' 'n' 'n' 'n' 'f' 's' 'n' 'r' 'r' 'n' 'f' 'n'
 'r' 'n' 's']
Wrong Labels [('f', 's'), ('f', 's'), ('f', 's'), ('n', 'r'), ('f', 'r'), ('f', 's')]
Training Features Shape: (185, 864)
Training Labels Shape: (185,)
Testing Features Shape: (21, 864)
Testing Labels Shape: (21,)
(185, 864)
(185,)


KeyboardInterrupt: 

In [11]:
wrong_df
wrong_df.groupby(wrong_df.columns.tolist(),as_index=False).size()

NameError: name 'wrong_df' is not defined

In [109]:
# 182: pellet goes up like a success?
# 240: pellet dispenser goes down during reach
# 83, 90: mouse drops pellet within/near acrylic, ambigious even to humans
# 

# Find Success, Fails, and Reaches

In [153]:
#Helper Functions
def get_predicted_times(classifier, df):
    dataframe = df[cols]
    inputs = []
    num_frames = frames_before + frames_after
    for index in range(num_frames, dataframe.shape[0] - num_frames, step_size):
        temp_df = dataframe.iloc[index - frames_before: index + frames_after]
        inputs.append(temp_df.values.flatten())
    inputs = np.asarray(inputs)
    predictions = classifier.predict(inputs)
    return predictions, list(range(num_frames, dataframe.shape[0] - num_frames, step_size))
def get_starts(lst):
    starts = [lst[0]]
    run = 0
    for i in range(1, len(lst)):
        run += 1
        if lst[i] - lst[i-1] > 0.2:
#             starts.append((lst[i], run))
            starts.append((lst[i]))
            run = 0
    return starts

#Find Reaches, 
def get_classified_starts(classifier, df):
    predictions2, start_frames = get_predicted_times(classifier, df)
    class_to_predictions = {}
    for start, prediction in zip(start_frames, predictions2):
        for c in classes:
            if prediction == c:
                if c in class_to_predictions.keys():
                    class_to_predictions[c].append(frame_to_time(start))
                else:
                    class_to_predictions[c] = [frame_to_time(start)]
    for c in classes:
        hold = get_starts(class_to_predictions[c])
        class_to_predictions[c] = hold
    return class_to_predictions
starts = get_classified_starts(attempt_classifier, test_df)
predicted_fails = starts['f']
predicted_reaches = starts['r']
predicted_successes = starts['s']
print(predicted_fails)
print(predicted_reaches)
print(predicted_successes)

[3.829, 6.181, 10.586, 11.011, 18.594, 20.996, 26.051, 30.631, 48.398, 52.778, 64.014]
[2.002, 2.828, 4.605, 6.006, 15.065, 15.666, 17.843, 19.595, 20.896, 26.777, 28.303, 30.606, 37.162, 38.238, 39.79, 40.766, 41.717, 43.493, 45.671, 46.597, 48.924, 50.325, 51.476, 58.634, 60.41, 60.886, 61.461, 62.362, 68.093, 69.244]
[18.694, 30.706, 52.878, 64.114]


In [154]:
x = sorted(predicted_fails + predicted_reaches + predicted_successes)
print(x)
print(len(x))

[2.002, 2.828, 3.829, 4.605, 6.006, 6.181, 10.586, 11.011, 15.065, 15.666, 17.843, 18.594, 18.694, 19.595, 20.896, 20.996, 26.051, 26.777, 28.303, 30.606, 30.631, 30.706, 37.162, 38.238, 39.79, 40.766, 41.717, 43.493, 45.671, 46.597, 48.398, 48.924, 50.325, 51.476, 52.778, 52.878, 58.634, 60.41, 60.886, 61.461, 62.362, 64.014, 64.114, 68.093, 69.244]
45


In [129]:
x = sorted(predicted_fails + predicted_reaches + predicted_successes)
print(x)
print(len(x))
import numpy as np
np.savetxt("file_name.csv", x, delimiter=",", fmt='%s', header='times')

[2.002, 2.578, 2.828, 3.829, 4.605, 5.981, 6.181, 10.586, 11.011, 15.04, 15.666, 17.843, 18.594, 18.694, 19.57, 20.896, 20.996, 26.026, 26.076, 26.777, 28.303, 30.581, 30.631, 30.706, 37.162, 38.213, 39.765, 40.791, 41.717, 43.468, 45.646, 46.171, 46.572, 48.373, 48.924, 50.35, 51.476, 52.778, 52.878, 58.634, 60.41, 60.886, 61.461, 62.337, 63.964, 64.114, 68.093, 69.244]
48
